<a href="https://colab.research.google.com/github/nptan2005/spark401_colab/blob/main/notebooks/Spark_basic_excercises_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧱 RESET ROADMAP – NHÁNH A

## HỌC TỪ GỐC: NumPy → Pandas → Spark

Mục tiêu giai đoạn này:

NHÌN DATA → BIẾT ĐANG TỐN GÌ → BIẾT SPARK SẼ LÀM GÌ

---

## 🔹 Phase A (A1 → A16): FOUNDATION & ARCHITECT MINDSET

|**Nhóm**|**Trạng thái**|
|--------|--------------|
|A1–A6|DataFrame / Lazy / Action / Partition|
|A7–A10|Join / Broadcast / Shuffle|
|A11–A14|Memory / Executor / Spark UI|
|A15|Shuffle & Skew (kẻ giết Spark)|
|A16|Partition Strategy (bài chốt)|


---

# 📘 PHASE A1 – NUMPY CĂN BẢN (KHÔNG ĐƯỢC BỎ)

❗ NumPy là “CPU” của Pandas & Spark
Không hiểu NumPy ⇒ không hiểu performance

---

🔹 BÀI 1.1 – NumPy Array là gì?

👉 Chạy ngay trên Colab

```python
import numpy as np

a = np.array([1, 2, 3, 4, 5])
a
```

### 🧠 Giải thích
* a không phải list Python
*	Là mảng số liên tục trong bộ nhớ (contiguous memory)
*	Mọi phép toán chạy bằng C, không phải Python loop

In [ ]:
import numpy as np

a = np.array([1, 2, 3, 4, 5])
a

array([1, 2, 3, 4, 5])

## 🔹 BÀI 1.2 – Vectorized operation (cực quan trọng)

```python
a * 10
```

###❓ Câu hỏi bắt buộc
*	Có vòng for không?
*	Vì sao nhanh?

### ✅ Trả lời đúng khi bạn hiểu:
*	Không loop Python
*	CPU xử lý cả mảng cùng lúc

In [ ]:
a * 10

array([10, 20, 30, 40, 50])

## 🔹 BÀI 1.3 – So sánh với Python loop (để “ngộ”)

```python
b = [1,2,3,4,5]

result = []
for x in b:
    result.append(x * 10)

result
```

### 🧠 Insight quan trọng

Spark & Pandas đều cố tránh Python loop giống NumPy


In [ ]:
b = [1,2,3,4,5]

result = []
for x in b:
    result.append(x * 10)

result

[10, 20, 30, 40, 50]

## 🔹 BÀI 1.4 – Shape & chiều dữ liệu

```python
a.shape
```

👉 (5,) = 1 chiều

```python
m = np.array([
    [1,2,3],
    [4,5,6]
])
m
```

👉 (2, 3) = 2 hàng, 3 cột

📌 DataFrame = 2D NumPy array + label

In [ ]:
a.shape

(5,)

In [ ]:
m = np.array([
    [1,2,3],
    [4,5,6]
])
m

m.shape

(2, 3)

## 🔹 BÀI 1.5 – Boolean mask (GỐC của filter)

```python
a > 2
```

```python
a[a > 2]
```

### 🔥 CỰC KỲ QUAN TRỌNG

* `df[df.amount > 100]` trong Pandas

* `df.filter(col("amount") > 100)` trong Spark

⬅️ bắt nguồn từ đây


In [ ]:
a > 2

a[a>2]

array([3, 4, 5])

# 📘 PHASE A2 – PANDAS (BẮT ĐẦU TỪ DATAFRAME)

---

## 🔹 BÀI 2.1 – Tạo DataFrame (cực basic)

```python
import pandas as pd

df = pd.DataFrame({
    "customer_id": ["1", "2", "3", "4"],
    "amount": [100, 200, 150, 300],
    "country": ["VN", "VN", "SG", "VN"]
})

df
```

👉 Hình dung: bảng Excel trong RAM

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "customer_id": ["1", "2", "3", "4"],
    "amount": [100, 200, 150, 300],
    "country": ["VN", "VN", "SG", "VN"]
})

df

,customer_id,amount,country
0,1,100,VN
1,2,200,VN
2,3,150,SG
3,4,300,VN


## 🔹 BÀI 2.2 – Select cột

```python
df["amount"]
```

```python
df[["customer_id", "amount"]]
```

Spark tương đương:

```python
df.select("customer_id", "amount")
```

In [ ]:
df["amount"]

df[["customer_id","amount"]]

,customer_id,amount
0,1,100
1,2,200
2,3,150
3,4,300


## 🔹 BÀI 2.3 – Filter (CỰC QUAN TRỌNG)

```python
df[df["amount"] > 150]
```

👉 Đây chính là Spark filter, không hơn không kém

In [ ]:
df[df["amount"] > 150]

,customer_id,amount,country
1,2,200,VN
3,4,300,VN


## 🔹 BÀI 2.4 – GroupBy (NGUỒN GỐC SHUFFLE)

```python
df.groupby("country")["amount"].sum()
```

### 🧠 DỪNG LẠI VÀ NGHĨ

*	Data được gom theo key
*	Mỗi key phải “tìm nhau”
*	Nếu data lớn → phải di chuyển data

👉 Spark gọi đây là shuffle


In [ ]:
df.groupby("country")["amount"].sum()

,amount
country,
SG,150
VN,600


## 🔹 BÀI 2.5 – Join (NGUỒN GỐC CỦA MỌI THẢM HỌA)

```python
customers = pd.DataFrame({
    "customer_id": ["1", "2", "3", "4"],
    "segment": ["MASS", "SME", "AFFLUENT", "MASS"]
})

df.merge(customers, on="customer_id", how="left")
```

🧠 Spark join = bản phân tán của dòng này

In [ ]:
customers = pd.DataFrame({
    "customer_id": ["1", "2", "3", "4"],
    "segment": ["MASS", "SME", "AFFLUENT", "MASS"]
})

df.merge(customers, on="customer_id", how="left")

,customer_id,amount,country,segment
0,1,100,VN,MASS
1,2,200,VN,SME
2,3,150,SG,AFFLUENT
3,4,300,VN,MASS


# ✅ ĐÁP ÁN CHUẨN – CHECKPOINT A1 & A2

## 🔹 A1 – NumPy

1. NumPy array khác Python list ở điểm nào?

### Trả lời chuẩn:
####	NumPy array:
*	Dữ liệu liên tục trong bộ nhớ
*	Cùng 1 kiểu dữ liệu
*	Tính toán bằng C / SIMD
####	Python list:
*	Chứa object pointer
*	Mỗi phần tử có thể khác type
*	Loop bằng Python → chậm

👉 Đây là lý do Spark & Pandas cố tránh Python loop

---

2. Vì sao NumPy nhanh?

Chuẩn:
*	Không dùng Python loop
*	CPU xử lý vectorized
*	Ít overhead object

---

3. Boolean mask liên quan gì đến filter Spark?

```python
a[a>2]
```

⬇️ chính là gốc của:

```python
df[df.amount > 100]        # Pandas
df.filter(col("amount") > 100)  # Spark
```

Spark chỉ phân tán hóa logic này

---

## 🔹 A2 – Pandas

1. GroupBy Pandas có giống Spark không?

Logic giống – cơ chế khác

|Pandas|Spark|
|------|-----|
|Chạy 1 máy|Chạy nhiều node|
|RAM local|RAM + Disk cluster|
|Không shuffle|Có shuffle|

👉 Nhưng bản chất toán học giống nhau

---

2. Join Pandas fail khi nào?
####	Khi:
*	Data lớn
*	RAM không đủ
*	Join key skew

👉 Spark ra đời để giải quyết điểm này

---

3. Spark giải quyết vấn đề gì Pandas không làm được?

Câu trả lời chuẩn:

> Spark giải quyết memory + scale, không phải logic





# 🚀 PHASE A3 – PANDAS PERFORMANCE & MEMORY

(Vì sao Pandas chết – Spark sống)

---

⚠️ PHASE NÀY CỰC KỲ QUAN TRỌNG
90% người học Spark không hiểu chỗ này

---

## 🧠 CÂU HỎI LỚN

> ❓ “Vì sao Pandas chạy tốt với 1M rows nhưng chết ở 20M?”

---

## 🔥 BÀI 3.1 – Pandas sống trong đâu?

👉 Pandas chỉ sống trong RAM

```text
Pandas DataFrame
= NumPy array
= RAM local
```

* ❌ Không spill disk
* ❌ Không phân tán
* ❌ Không retry

---

### 🔹 Demo (conceptual – không cần crash Colab)

Giả sử:

```python
df = pd.read_csv("orders_50M.csv")
```

**Nếu:**
*	File 5 GB
*	RAM còn 3 GB

👉 CRASH NGAY

---

## 🔥 BÀI 3.2 – Vì sao Pandas OOM rất nhanh

#### Nguyên nhân 1: Object dtype

```python
df["customer_id"].dtype
```

Nếu là:

```text
object
```

👉 mỗi cell là 1 Python object

#### Nguyên nhân 2: Copy ngầm (silent copy)

```python
df2 = df[df.amount > 100]
```

**👉 Pandas:**
*	Tạo DataFrame mới
*	Copy data

⛔ RAM tăng gấp đôi mà bạn không biết

---

#### Nguyên nhân 3: GroupBy & Join

```python
df.groupby("customer_id").sum()
```

**👉 Pandas phải:**
*	Giữ toàn bộ data
*	Giữ hash map
*	Giữ result

🔥 RAM nổ rất nhanh

---

## 🔥 BÀI 3.3 – Spark khác Pandas ở điểm nào?

#### 🧠 Spark KHÔNG load toàn bộ data

|**Pandas**|**Spark**|
|----------|---------|
|Load hết|Load theo partition|
|RAM local|RAM + Disk|
|Copy nhiều|Pipeline|
|OOM là chết|OOM có thể spill|

---

#### Spark DataFrame =

```text
Logical plan
+ partitions
+ lazy execution
```

👉 Data chỉ được đọc khi action

---

### 🔹 Ví dụ so sánh trực tiếp

#### Pandas

```python
df = pd.read_csv("big.csv")
df.groupby("country").sum()
```

❌ Load hết → OOM

---

#### Spark

```python
df = spark.read.csv("big.csv")
df.groupBy("country").sum().show()
```

#### ✅
*	Đọc từng partition
*	Shuffle có kiểm soát
*	Spill nếu cần

---

## 🔥 BÀI 3.4 – Vậy Spark có OOM không?

#### 👉 CÓ, khi:
*	Broadcast sai
*	Skew nặng
*	Cache bừa
*	Executor quá to
*	Join design sai

➡️ Spark không phải thuốc tiên

---







# PHASE A4 – Spark từ con số 0, đúng như cam kết:

❗ Không tối ưu
❗ Không shuffle / AQE / join strategy
✅ Chỉ hiểu Spark vận hành như thế nào

---

# 🚀 PHASE A4 – SPARK CƠ BẢN (NHƯNG ĐÚNG TƯ DUY)

## 🎯 Mục tiêu A4

Sau phase này, bạn phải trả lời được bằng bản năng:

*	Spark DataFrame là gì thật sự?
*	Partition đang nằm ở đâu?
*	Vì sao code chạy mà chưa đọc dữ liệu?
*	explain() đọc chỗ nào trước, bỏ chỗ nào

---

## 🔥 BÀI 4.1 – Spark DataFrame là gì?

#### ❌ Cách hiểu sai (rất phổ biến)

Spark DataFrame = dữ liệu

> **⛔ SAI**

---

#### ✅ Cách hiểu đúng

Spark DataFrame = KẾ HOẠCH XỬ LÝ DỮ LIỆU

**Nó chỉ là:**

*	Logical plan
*	Chưa có data
*	Chưa tốn RAM
*	Chưa chạy gì cả



### 🧠 So sánh cực quan trọng

|Pandas|Spark|
|------|------|
|Data nằm trong df|	Data chưa đọc|
|df = dữ liệu	|df = kế hoạch|
df xem là nặng|	df xem là nhẹ|


---

## 🔥 BÀI 4.2 – Demo “Spark chưa chạy gì cả”

Code

```python
df = spark.read.csv("/content/sample_data/california_housing_test.csv", header=True)
```

👉 Lúc này:

*	❌ Chưa đọc file
*	❌ Chưa load RAM
*	❌ Chưa có task

---

Kiểm tra

```python
print(df)
```

Bạn sẽ thấy:

```python
DataFrame[longitude: string, latitude: string, ...]
```

👉 KHÔNG CÓ DATA

---

## 🔥 BÀI 4.3 – Action là gì?

> Action = lệnh BẮT SPARK PHẢI CHẠY

**Ví dụ action:**

```python
df.show()
df.count()
df.collect()
```

---

Ví dụ rất rõ

```python
df2 = df.filter("median_house_value > 100000")
```

* ❌ Chưa chạy
* ❌ Chưa đọc file

---

```python
df2.count()
```

🔥 BÙM – Spark mới bắt đầu làm việc

---

## 🔥 BÀI 4.4 – Partition là gì?

**Câu trả lời chuẩn:**

> Partition = đơn vị xử lý song song nhỏ nhất của Spark

---

Hình dung

```text
File CSV
↓
Spark chia thành nhiều partition
↓
Mỗi partition = 1 task
```

---

Kiểm tra số partition

```python
df.rdd.getNumPartitions()
```

---

Ví dụ

```python
df = spark.read.csv("big.csv")
print(df.rdd.getNumPartitions())
```

👉 Spark tự quyết định, bạn chưa cần chỉnh

---

## 🔥 BÀI 4.5 – Lazy execution (điểm ăn tiền nhất)

Ví dụ

```python
df = spark.read.csv("file.csv")
df2 = df.filter("price > 100")
df3 = df2.select("price")
```

**👉 Spark ghi nhớ:**

*	Read
*	Filter
*	Select

❌ Chưa làm gì cả

---

Khi action xảy ra

```python
df3.show()
```


Spark sẽ:

1.	Gom toàn bộ logic
2.	Tối ưu
3.	Chạy 1 pipeline

👉 KHÔNG chạy từng dòng code

---

## 🔥 BÀI 4.6 – explain(): Đọc sao cho không loạn?

Code

```python
df3.explain()
```

Bạn sẽ thấy đại loại:

```text
== Physical Plan ==
*(1) Project
+- *(1) Filter
   +- FileScan csv
```

---

Cách đọc ĐÚNG (rất quan trọng)

#### 👉 Đọc từ dưới lên

1️⃣ FileScan → đọc dữ liệu

2️⃣ Filter → lọc

3️⃣ Project → chọn cột

---

#### ❌ Đừng làm

*	Đừng đọc cost
*	Đừng đọc AQE
*	Đừng đọc Broadcast

⛔ Chưa tới lúc

---

## 🧠 CHECKPOINT A4 – BẮT BUỘC HIỂU

Bạn đã hiểu đúng nếu trả lời được:

1.	Spark DataFrame có dữ liệu chưa?
2.	Vì sao filter() không chạy?
3.	Khi nào Spark mới đọc file?
4.	Partition là đơn vị của cái gì?
5.	explain() đọc từ trên xuống hay dưới lên?




# PHASE A5 – TRANSFORMATION vs ACTION (CỐT LÕI SỐNG CÒN CỦA SPARK)

⚠️ Phase này cực kỳ quan trọng

Nếu A5 chưa vững → học tuning, join, shuffle chỉ là học vẹt

---

# 🚀 PHASE A5 – TRANSFORMATIONS vs ACTIONS

## 🎯 Mục tiêu A5

Sau phase này, bạn phải nhìn code là biết:

*	Dòng nào chưa chạy
*	Dòng nào bắt Spark chạy
*	Vì sao collect() nguy hiểm
*	Vì sao count() thường an toàn hơn

---

## 🔥 BÀI 5.1 – Transformation là gì?

### Định nghĩa chuẩn

Transformation = tạo kế hoạch mới từ kế hoạch cũ

* 👉 KHÔNG chạy
* 👉 KHÔNG đọc data
* 👉 KHÔNG tốn RAM

---

Ví dụ

```python
df = spark.read.csv("data.csv", header=True)
```

👉 Transformation (read)

```python
df2 = df.filter("price > 100")
```

👉 Transformation (filter)

```python
df3 = df2.select("price")
```

> 👉 Transformation (select)

⛔ Chưa có dòng nào chạy

---

## 🔥 BÀI 5.2 – Action là gì?

### Định nghĩa chuẩn

Action = yêu cầu Spark TRẢ KẾT QUẢ về Driver

#### 🔥 Spark buộc phải chạy

---

Các action phổ biến

```python
df.show()
df.count()
df.collect()
df.take(10)
df.write.parquet(...)
```

---

Ví dụ sống còn

```python
df3.count()
```

👉 Spark:

1.	Đọc file
2.	Filter
3.	Select
4.	Đếm
5.	Trả về 1 số

---

## 🔥 BÀI 5.3 – Vì sao collect() nguy hiểm?

Code

```python
data = df3.collect()
```

👉 Điều gì xảy ra?

*	Spark chạy
*	Toàn bộ data kéo về DRIVER
*	RAM driver chết nếu data lớn

---

### So sánh

|**Lệnh**|**An toàn?**|**Vì sao**|
|--------|------------|----------|
|count()|	✅|	Trả về 1 số|
|show()	|⚠️	|Mặc định 20 dòng|
|take(10)|	⚠️|	Chỉ 10 dòng|
|collect()	|❌	|Kéo hết data|

---

## 🔥 BÀI 5.4 – Demo cực rõ (Colab chạy được)

```python
df = spark.range(0, 1_000_000)
```

### Case 1 – OK

```python
df.count()
```

👉 Trả về 1000000

---

### Case 2 – Cẩn thận

```python
df.show()
```

👉 OK (20 dòng)

---

### Case 3 – ❌ Không làm

```python
df.collect()
```

👉 Với data lớn → driver OOM

---

## 🔥 BÀI 5.5 – map vs select (Spark style)

### ❌ Không Spark-style

```python
df.rdd.map(lambda x: x[0] * 2)
```

* 👉 Chuyển sang RDD
* 👉 Mất optimizer
* 👉 Chậm

---

### ✅ Spark-style

```python
from pyspark.sql.functions import col

df2 = df.select((col("id") * 2).alias("id2"))
```

* 👉 Catalyst hiểu
* 👉 Tối ưu được

---

## 🔥 BÀI 5.6 – withColumn (dùng đúng cách)

Ví dụ

```python
df2 = df.withColumn("id2", col("id") * 2)
```

* 👉 Transformation
* 👉 Chưa chạy

---

### Sai lầm phổ biến

```python
for i in range(10):
    df = df.withColumn(f"c{i}", col("id") + i)
```

* ⛔ Tạo 10 kế hoạch lồng nhau
* ⛔ Code chạy được nhưng plan rất nặng

---

## 🔥 BÀI 5.7 – explain() gắn với A5

```python
df3.explain()
```

Bạn sẽ thấy:

```text
== Physical Plan ==
*(1) Project
+- *(1) Filter
   +- Range
```

👉 Không có action → chưa chạy

---

## 🧠 CHECKPOINT A5 – BẮT BUỘC TRẢ LỜI ĐƯỢC

Hãy tự trả lời (không cần gửi):

1.	Vì sao filter() không chạy?
2.	Vì sao count() an toàn hơn collect()?
3.	show() có phải action không?
4.	Vì sao dùng DataFrame API tốt hơn RDD?
5.	Khi nào Spark mới thực sự đọc data?


In [ ]:
!apt-get install -y openjdk-17-jdk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  at-spi2-core fonts-dejavu-core fonts-dejavu-extra gsettings-desktop-schemas
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common librsvg2-common libxcomposite1 libxt-dev
  libxtst6 libxxf86dga1 openjdk-17-jre session-migration x11-utils
Suggested packages:
  gvfs libxt-doc openjdk-17-demo openjdk-17-source visualvm mesa-utils
The following NEW packages will be installed:
  at-spi2-core fonts-dejavu-core fonts-dejavu-extra gsettings-desktop-schemas
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common librsvg2-common libxcomposite1 libxt-dev
  libxtst6 libxxf86dga1 openjdk-17-jdk openjdk-17-jr

In [ ]:
!wget https://archive.apache.org/dist/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz
!tar xf spark-4.0.1-bin-hadoop3.tgz

--2026-01-07 08:31:51--  https://archive.apache.org/dist/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 548955321 (524M) [application/x-gzip]
Saving to: ‘spark-4.0.1-bin-hadoop3.tgz’

spark-4.0.1-bin-had 100%[===================>] 523.52M  17.1MB/s    in 45s     

2026-01-07 08:32:36 (11.7 MB/s) - ‘spark-4.0.1-bin-hadoop3.tgz’ saved [548955321/548955321]



In [ ]:
# ===============================
# Spark 4.0.1 Setup (REQUIRED)
# ===============================
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-4.0.1-bin-hadoop3"
os.environ["PATH"] += ":/content/spark-4.0.1-bin-hadoop3/bin"

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark401-Training") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

print("Spark version:", spark.version)

Spark version: 4.0.1


In [ ]:
df = spark.range(0, 1_000_000)

In [ ]:
# Case 1: count
df.count()

1000000

In [ ]:
# Case 2: Show
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+
only showing top 20 rows


In [ ]:
# Case 3: test Driver OOM
df.collect()

[Row(id=0),
 Row(id=1),
 Row(id=2),
 Row(id=3),
 Row(id=4),
 Row(id=5),
 Row(id=6),
 Row(id=7),
 Row(id=8),
 Row(id=9),
 Row(id=10),
 Row(id=11),
 Row(id=12),
 Row(id=13),
 Row(id=14),
 Row(id=15),
 Row(id=16),
 Row(id=17),
 Row(id=18),
 Row(id=19),
 Row(id=20),
 Row(id=21),
 Row(id=22),
 Row(id=23),
 Row(id=24),
 Row(id=25),
 Row(id=26),
 Row(id=27),
 Row(id=28),
 Row(id=29),
 Row(id=30),
 Row(id=31),
 Row(id=32),
 Row(id=33),
 Row(id=34),
 Row(id=35),
 Row(id=36),
 Row(id=37),
 Row(id=38),
 Row(id=39),
 Row(id=40),
 Row(id=41),
 Row(id=42),
 Row(id=43),
 Row(id=44),
 Row(id=45),
 Row(id=46),
 Row(id=47),
 Row(id=48),
 Row(id=49),
 Row(id=50),
 Row(id=51),
 Row(id=52),
 Row(id=53),
 Row(id=54),
 Row(id=55),
 Row(id=56),
 Row(id=57),
 Row(id=58),
 Row(id=59),
 Row(id=60),
 Row(id=61),
 Row(id=62),
 Row(id=63),
 Row(id=64),
 Row(id=65),
 Row(id=66),
 Row(id=67),
 Row(id=68),
 Row(id=69),
 Row(id=70),
 Row(id=71),
 Row(id=72),
 Row(id=73),
 Row(id=74),
 Row(id=75),
 Row(id=76),
 Row(id=7

In [ ]:
df.rdd.map(lambda x: x[0] * 2)

PythonRDD[23] at RDD at PythonRDD.scala:56

In [ ]:
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+
only showing top 20 rows


In [ ]:
from pyspark.sql.functions import col

df2 = df.select((col("id") * 2).alias("id2"))

In [ ]:
df2.show()

+---+
|id2|
+---+
|  0|
|  2|
|  4|
|  6|
|  8|
| 10|
| 12|
| 14|
| 16|
| 18|
| 20|
| 22|
| 24|
| 26|
| 28|
| 30|
| 32|
| 34|
| 36|
| 38|
+---+
only showing top 20 rows


In [ ]:
df2 = df.withColumn("id2", col("id") * 2)

In [ ]:
df2.explain()

== Physical Plan ==
*(1) Project [id#0L, (id#0L * 2) AS id2#23L]
+- *(1) Range (0, 1000000, step=1, splits=2)





# 📘 PHASE A6 – GROUP BY & AGGREGATION (SPARK CORE)

## 🎯 Mục tiêu A6
*	Hiểu groupBy() thực sự làm gì
*	Phân biệt aggregation logic vs execution
*	Biết vì sao groupBy luôn là điểm nguy hiểm trong Spark
*	Chưa nói tuning – chỉ hiểu bản chất

---

## 1️⃣ GROUP BY trong Spark là gì?

### 📌 Định nghĩa chuẩn

**groupBy** = gom các dòng có cùng key để tính toán chung

#### Ví dụ:
* Tổng tiền theo customer_id
*	Số giao dịch theo country
*	KPI theo date

---

### 🔍 Ví dụ cực cơ bản

```python
from pyspark.sql import functions as F

df = spark.createDataFrame(
    [
        ("A", 10),
        ("A", 20),
        ("B", 5),
        ("B", 15),
    ],
    ["key", "value"]
)

df.show()
```

Kết quả:

|key|	value|
|---|------|
|A|	10|
|A	|20|
|B	|5|
|B	|15|


---

## 2️⃣ groupBy() chưa chạy

```python
agg_df = df.groupBy("key").sum("value")
```

* 👉 Đây là TRANSFORMATION
* 👉 Spark chưa đọc data
* 👉 Chỉ tạo Logical Plan

---

### Khi nào Spark chạy?

```python
agg_df.show()
```

* 👉 Đây là ACTION

* 👉 Spark mới thực sự:
1.	Đọc dữ liệu
2.	Gom nhóm
3.	Tính toán
4.	Trả kết quả

---

## 3️⃣ Kết quả aggregation

```python
agg_df.show()
```

|key|	sum(value)|
|---|-----------|
|A	|30|
|B	|20|


---

## 4️⃣ Vì sao groupBy “nguy hiểm”?

### ❗ Vấn đề cốt lõi

#### Các dòng có cùng key
👉 ban đầu nằm rải rác ở nhiều partition

#### Spark BUỘC phải:
*	Gom các key giống nhau về cùng nơi
*	Rồi mới tính toán được

📌 Đây là lý do groupBy rất nặng

---

## 5️⃣ explain() để nhìn bản chất

```python
agg_df.explain()
```

Bạn sẽ thấy (rút gọn):

```text
== Physical Plan ==
HashAggregate
+- Exchange
   +- HashAggregate
      +- Scan
```

### Giải thích từng dòng

|**Thành phần**|**Ý nghĩa**|
|--------------|-----------|
|Scan	|Đọc dữ liệu|
|HashAggregate (local)|	Gom tạm trong partition|
|Exchange	|🚨 Đổi dữ liệu giữa các executor|
|HashAggregate (global)|	Gom cuối|


👉 Exchange = điểm đắt nhất

---

## 6️⃣ Những aggregation thường gặp

```python
df.groupBy("key").count()
df.groupBy("key").sum("value")
df.groupBy("key").avg("value")
df.groupBy("key").max("value")
```

Hoặc nhiều metric:

```python
df.groupBy("key").agg(
    F.sum("value").alias("total"),
    F.count("*").alias("cnt")
)
```

---

## 7️⃣ groupBy nhiều cột

```python
df.groupBy("country", "channel").sum("amount")
```

### 📌 Càng nhiều key:
*	Cardinality càng cao
*	Output càng lớn
*	Aggregation càng nặng

---

## 8️⃣ LỖI TƯ DUY RẤT PHỔ BIẾN

### ❌ Nghĩ sai

***“Data nhỏ nên groupBy thoải mái”***

### ❗ Sai vì:
*	Spark chạy phân tán
*	Shuffle cost ≠ size file
*	Key distribution mới là vấn đề

---

### ❌ Nghĩ sai

***“groupBy chỉ là SQL”***

### ❗ Sai vì:
*	groupBy = network + memory + disk
*	Không chỉ là logic

---

## 9️⃣ Bank-grade mindset (rất quan trọng)

### Trong ngân hàng:
*	❌ Không groupBy lung tung
*	❌ Không groupBy trên raw data
*	✅ Aggregate từng layer
*	✅ Silver → chuẩn hoá
*	✅ Gold → KPI nhẹ

---

## 🧠 CHECKPOINT A6 – PHẢI NHỚ

*	groupBy() là transformation
*	show()/count() mới chạy
*	groupBy luôn cần gom dữ liệu
*	Exchange = dấu hiệu nguy hiểm
*	Aggregation là điểm tốn tiền nhất


In [ ]:
from pyspark.sql import functions as F

df = spark.createDataFrame(
    [
        ("A", 10),
        ("A", 20),
        ("B", 5),
        ("B", 15),
    ],
    ["key", "value"]
)

df.show()

+---+-----+
|key|value|
+---+-----+
|  A|   10|
|  A|   20|
|  B|    5|
|  B|   15|
+---+-----+



In [ ]:
# groupby nhưng chưa chạy
agg_df = df.groupBy("key").sum("value")

In [ ]:
# Chạy
agg_df.show()

+---+----------+
|key|sum(value)|
+---+----------+
|  A|        30|
|  B|        20|
+---+----------+



In [ ]:
agg_df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[key#24], functions=[sum(value#25L)])
   +- Exchange hashpartitioning(key#24, 4), ENSURE_REQUIREMENTS, [plan_id=162]
      +- HashAggregate(keys=[key#24], functions=[partial_sum(value#25L)])
         +- Scan ExistingRDD[key#24,value#25L]




# PHASE A7 – SHUFFLE (KẺ GIẾT SPARK JOB)

---

# 📕 PHASE A7 – SHUFFLE (SPARK CORE – HIỂU ĐÚNG, KHÔNG SỢ)

## 🎯 Mục tiêu A7
*	Hiểu shuffle là gì (đúng nghĩa vật lý)
*	Vì sao shuffle không tránh được
*	Vì sao shuffle giết performance + cost
*	Nhìn explain() là biết có shuffle hay không

---

## 1️⃣ SHUFFLE LÀ GÌ? (NÓI THEO NGƯỜI ĐI LÀM)

**Shuffle** = Spark phải chuyển dữ liệu giữa các executor

### Không phải:
*	Không phải SQL keyword
*	Không phải function

👉 Shuffle là hành động vật lý

---

## 📦 Ví dụ rất đời

### Giả sử:
*	4 executor
*	Mỗi executor giữ 1 phần data

```text
Executor 1: A, B
Executor 2: B, C
Executor 3: A, C
Executor 4: A, B
```

### Nếu bạn làm:

```python
df.groupBy("key").sum("value")
```

### 👉 Spark buộc phải:
*	Gom tất cả A về 1 nơi
*	Gom tất cả B về 1 nơi
*	Gom tất cả C về 1 nơi

> 📌 Không làm vậy → kết quả sai

➡️ Việc “gom lại” đó chính là SHUFFLE

---

## 2️⃣ Shuffle xảy ra khi nào?

### 🚨 Các operation LUÔN shuffle

|**Operation**|**Lý do**|
|-------------|---------|
|groupBy	|Gom key|
|join (non-broadcast)|	Match key|
|distinct	|Loại trùng|
|orderBy|	Sắp xếp toàn cục|
|repartition	|Đổi phân vùng«


---

### ❌ Các operation KHÔNG shuffle

|**Operation**|**Vì sao**|
|-------------|----------|
|select	|Chỉ đổi cột|
|withColumn|	Tính toán trong row|
|filter	|Loại row|
|limit (cục bộ)|	Không gom key|
|coalesce|	Gộp partition, không đổi dữ liệu|


---

## 3️⃣ Shuffle tốn gì?

Shuffle KHÔNG miễn phí, nó tốn:

### 🔥 1. Network
*	Executor gửi data cho executor khác
*	Chậm, dễ timeout

### 🔥 2. Disk
*	Shuffle file ghi ra disk
*	Disk IO rất đắt

### 🔥 3. Memory
*	Buffer shuffle
*	Dễ spill to disk

### 🔥 4. Time
*	Chờ task khác
*	Gây stage stuck 99%

---

## 4️⃣ explain(): nhìn là biết shuffle

Ví dụ:

```python
df.groupBy("key").sum("value").explain()
```

Nếu bạn thấy:

```text
Exchange hashpartitioning(key)
```

👉 CHẮC CHẮN có shuffle

---

### 📌 Rule nhớ suốt đời

CÓ Exchange = CÓ SHUFFLE

> Không cần hiểu sâu plan, chỉ cần nhớ dòng này.

---

## 5️⃣ Shuffle ≠ Data lớn

🚨 Rất nhiều người hiểu sai:

“Data nhỏ thì shuffle không sao”

### ❌ Sai

#### Vì sao?
*	Data nhỏ nhưng:
*	Key skew
*	Partition lệch
*	Executor imbalance

👉 Shuffle vẫn rất nặng

---

## 6️⃣ Ví dụ thực tế (Bank-grade)

### ❌ Sai

```python
# Raw transaction
tx.groupBy("customer_id").sum("amount")
```

* 👉 Hàng triệu customer
* 👉 Shuffle khổng lồ
* 👉 Job chậm / fail

---

### ✅ Đúng

```python
# Silver layer: aggregate theo ngày
tx.groupBy("customer_id", "txn_date").sum("amount")

# Gold layer: aggregate lại
daily.groupBy("customer_id").sum("amount")
```

* 👉 Shuffle chia nhỏ theo layer
* 👉 Kiểm soát được

---

## 7️⃣ Shuffle và SLA (cực quan trọng)

### Trong bank:
* Shuffle = không predictable
*	Không predict được:
*	Runtime
*	Cost
*	Failure

👉 Bank rất ghét shuffle không kiểm soát

---

## 8️⃣ LỖI TƯ DUY PHỔ BIẾN

### ❌ Nghĩ sai

“Tăng executor là xong”

#### ❗ Không:
*	Shuffle vẫn phải xảy ra
*	Network vẫn là bottleneck

---

### ❌ Nghĩ sai

“Tune config sẽ cứu”

#### ❗ Không:
*	Shuffle là design problem
*	Config chỉ cứu 5–10%

---

## 9️⃣ CHECKPOINT A7 – BẮT BUỘC NHỚ

*	Shuffle = chuyển data giữa executor
*	Shuffle là vật lý, không phải logic
*	Exchange = shuffle
*	Shuffle tốn network + disk + memory
*	Shuffle là kẻ giết Spark job



# PHASE A8 – PARTITION (CỐT LÕI ĐỂ ĐIỀU KHIỂN SPARK)

---

# 📕 PHASE A8 – PARTITION (VÌ SAO SPARK CHẬM DÙ DATA NHỎ)

## 🎯 Mục tiêu A8
* Hiểu partition là gì (đúng nghĩa vật lý)
*	Vì sao partition quyết định số task
*	Vì sao partition sai → shuffle nặng
*	Phân biệt repartition vs coalesce
*	Biết nhìn partition để predict runtime

---

## 1️⃣ PARTITION LÀ GÌ?

**Partition** = đơn vị công việc nhỏ nhất mà Spark xử lý song song

*	1 partition → 1 task
*	1 task → chạy trên 1 core

* 📌 Spark không chạy theo row
* 📌 Spark chạy theo partition

---

### 🧠 Hình dung rất đời

#### Giả sử bạn có:
*	1000 dòng data
*	Nhưng chỉ 1 partition

>* 👉 Spark chỉ dùng 1 core
>* 👉 15 core còn lại ngồi chơi

---

## 2️⃣ Partition ≠ File

### 🚨 Sai lầm rất phổ biến:

“File nhiều → partition nhiều”

### ❌ Không đúng
*	File chỉ là input
*	Partition là logical split trong Spark

Ví dụ:

```python
df = spark.read.parquet("data/")
```

*	10 file
*	Nhưng Spark có thể tạo:
*	10 partition
*	hoặc 1 partition
*	hoặc 200 partition

> ➡️ Do Spark quyết định

---

## 3️⃣ Số partition ảnh hưởng gì?

### 📌 Công thức sống còn

Số task = số partition


|**Partition**|**Hậu quả**|
|-------------|-----------|
|Quá ít|	Không tận dụng CPU|
|Quá nhiều|	Overhead, scheduler mệt|
|Lệch|	Task chạy lâu → stage stuck|


---

## 4️⃣ Partition và Shuffle liên quan thế nào?

### 🔥 Trước shuffle
*	Data có partition A

### 🔥 Sau shuffle
* Spark repartition lại toàn bộ data

> 👉 Shuffle luôn tạo partition mới

---

#### Ví dụ

```python
df.groupBy("key").count()
```

#### 👉 Spark làm:
1.	Shuffle data theo key
2.	Tạo spark.sql.shuffle.partitions partition

#### Mặc định:

```python
spark.sql.shuffle.partitions = 200
```

#### 🚨 Với dataset nhỏ → rất nguy hiểm

---

## 5️⃣ Vì sao 200 partition là “sát thủ”?

### Giả sử:
* Data chỉ 50MB
*	200 partition

> 👉 Mỗi partition ~250KB

### Hậu quả:
*	Scheduler overhead lớn
*	Task setup > task work
*	Job chậm hơn cả Pandas

---

## 6️⃣ Kiểm tra partition

Xem số partition

```python
df.rdd.getNumPartitions()
```

Xem partition sau shuffle

```python
df.groupBy("key").count().rdd.getNumPartitions()
```

---

## 7️⃣ repartition vs coalesce (rất hay nhầm)

### 🔴 repartition

```python
df.repartition(8)
```

*	Có shuffle
*	Phân phối lại data đều
*	Dùng khi:
-	Join
-	GroupBy
-	Data skew

---

### 🟢 coalesce

```python
df.coalesce(8)
```

*	Không shuffle
*	Chỉ gộp partition
*	Có thể bị lệch
*	Dùng khi:
-	Reduce output
-	Trước write

---

### 📌 Quy tắc vàng

#### Mục đích	Dùng
* Chia đều lại data	repartition
* Giảm partition	coalesce


---

## 8️⃣ Partition tốt trông như thế nào?

### ✅ Tốt
*	Task time gần bằng nhau
*	Không task nào vượt trội
*	Stage kết thúc nhanh

### ❌ Xấu
*	1–2 task chạy rất lâu
*	Stage stuck 99%
*	CPU rảnh nhưng job chưa xong

---

## 9️⃣ Partition và SLA

### Trong bank:
*	Partition sai → runtime không predict
*	Không predict → vi phạm SLA
*	SLA vỡ → incident

> 👉 Partition là design decision, không phải tuning nhỏ.

---

## 🔟 CHECKPOINT A8 – PHẢI NHỚ
*	Partition = task
*	Task = core
*	Partition sai → Spark chậm
*	Shuffle luôn tạo partition mới
*	repartition có shuffle
*	coalesce không shuffle


# PHASE A9 – GROUP BY & AGGREGATION (KẺ GIẾT SPARK THẦM LẶNG)

---

# 📕 PHASE A9 – GROUP BY & AGGREGATION

## 🎯 Sau A9, bạn phải trả lời được:
*	Vì sao groupBy() luôn gây shuffle
*	Vì sao 1 dòng code có thể làm job từ 5 phút → 3 giờ
*	Khi nào aggregation an toàn, khi nào cực kỳ nguy hiểm
*	Cách thiết kế aggregation kiểu bank-grade

---

## 1️⃣ SỰ THẬT SỐC: groupBy() = SHUFFLE 100%

Không có ngoại lệ.

```python
df.groupBy("key").count()
```

#### Spark BẮT BUỘC:
1.	Thu thập tất cả record có cùng key
2.	Đưa chúng về cùng 1 executor
3.	Tạo Exchange (shuffle)

#### 📌 Vì:
*	Spark không biết record nào thuộc key nào nếu không shuffle

---

## 2️⃣ groupBy nguy hiểm ở đâu?

### 🔥 Nguy hiểm #1 – Shuffle toàn bộ data
*	1TB input
*	groupBy 1 cột

## 👉 1TB bị:
*	Serialize
*	Network transfer
*	Deserialize

---

### 🔥 Nguy hiểm #2 – Skew (key không đều)

#### Ví dụ:
*	1 customer = 30% dữ liệu
*	999 customer còn lại = 70%

* 👉 1 task xử lý 30%
* 👉 Các task khác xong sớm
* 👉 Stage stuck 99%

---

### 🔥 Nguy hiểm #3 – Memory pressure

#### Aggregation cần:
*	Giữ state (map, hash table)
*	Lưu intermediate result

#### 👉 Dễ:
*	Spill to disk
*	OOM

---

## 3️⃣ Ví dụ đơn giản nhưng chí mạng

```python
orders.groupBy("customer_id").sum("amount")
```

### Câu hỏi kiến trúc sư phải hỏi, không phải coder:
*	Có bao nhiêu customer?
*	Distribution có đều không?
*	Có hot key không?
*	SLA bao lâu?

---

## 4️⃣ Vì sao GROUP BY KHÁC FILTER?

|**filter**|**groupBy**|
|----------|-----------|
|Không shuffle|	Luôn shuffle|
|Chạy song song tốt|Dễ skew|
|Ít memory|	Tốn memory|

* 👉 filter = an toàn
* 👉 groupBy = nguy hiểm

---

## 5️⃣ spark.sql.shuffle.partitions và groupBy

Sau groupBy, Spark tạo:

```python
spark.sql.shuffle.partitions = 200
```

#### Nghĩa là:
*	200 reducer
*	200 task aggregation

🚨 Với dataset nhỏ → overhead
🚨 Với dataset lớn → nếu skew vẫn chết

---

## 6️⃣ Khi nào groupBy được xem là “AN TOÀN”?

### ✅ Tương đối an toàn khi:
*	Key phân bố đều
*	Cardinality cao
*	Không có hot key
*	Data đã được pre-aggregated

#### Ví dụ:

```python
groupBy("date", "country")
```

---

### ❌ RẤT NGUY HIỂM khi:
*	groupBy customer_id
*	groupBy account_id
*	groupBy merchant_id
*	groupBy user_id

(đặc biệt trong bank 😈)

---

## 7️⃣ Bank-grade aggregation: TƯ DUY ĐÚNG

### ❌ Sai (coder mindset)

```python
orders.groupBy("customer_id").sum("amount")
```

---

### ✅ Đúng (architect mindset)

Bước 1 – Pre-aggregation (Silver)

```python
orders.groupBy("customer_id", "date").sum("amount")
```

Bước 2 – Gold aggregation (nhẹ)

```python
daily.groupBy("customer_id").sum("amount")
```

* 👉 Chia nhỏ aggregation
* 👉 Giảm shuffle size
* 👉 Predict runtime

---

## 8️⃣ groupBy vs reduceByKey (RDD)

Kiến thức nền, để hiểu sâu hơn

|**groupByKey**|**reduceByKey**|
|--------------|---------------|
|Shuffle full values|	Combine trước|
|Rất tốn memory	|Hiệu quả hơn|
|Ít dùng|	Tư duy tốt|

👉 DataFrame đã tối ưu hơn RDD, nhưng tư duy này vẫn đúng

---

## 9️⃣ Cách đọc execution plan với groupBy

```python
df.groupBy("k").count().explain()
```

### Bạn PHẢI thấy:
* Exchange
*	HashAggregate / SortAggregate


Nếu không thấy → bạn đọc plan sai.

---

## 🔟 CHECKPOINT A9 – PHẢI THUỘC
*	groupBy = shuffle 100%
*	Shuffle = network + disk + memory
*	groupBy key sai = job chết
*	Bank-grade aggregation = chia nhỏ
*	Gold layer chỉ aggregate nhẹ


# PHASE A10 – JOIN (ÁC MỘNG THẬT SỰ CỦA SPARK)

---

# 📕 PHASE A10 – JOIN TRONG SPARK (BANK-GRADE)

## 🎯 Mục tiêu A10

### Sau bài này, bạn phải:
*	Nhìn join() là ngửi thấy shuffle
*	Phân biệt JOIN an toàn vs JOIN tự sát
*	Đọc explain() và biết Spark chọn strategy gì
*	Biết khi nào TUYỆT ĐỐI không được join

---

## 1️⃣ SỰ THẬT PHŨ PHÀNG

JOIN = SHUFFLE + MEMORY + RỦI RO

Không có join “nhẹ” nếu bạn không kiểm soát.

---

## 2️⃣ Join làm Spark làm gì?

#### Ví dụ:

```pythin
orders.join(customers, "customer_id")
```

#### Spark BUỘC PHẢI:
1.	Đưa các record có cùng customer_id về cùng executor
2.	Ghép row
3.	Trả kết quả

> 👉 => SHUFFLE

---

## 3️⃣ 4 JOIN STRATEGIES trong Spark (CẦN THUỘC)

### 1️⃣ BroadcastHashJoin (THIÊN THẦN 😇)

```python
from pyspark.sql.functions import broadcast

orders.join(broadcast(customers), "customer_id")
```

#### Điều kiện:
* 1 bảng nhỏ
*	Fit trong memory executor

#### Ưu điểm:
*	❌ Không shuffle bảng lớn
*	⚡ Rất nhanh

#### Nguy hiểm nếu dùng sai:
*	OOM
*	Kill executor
*	Job fail toàn cluster

---

### 2️⃣ SortMergeJoin (ÁC QUỶ 😈 – phổ biến nhất)

#### Spark dùng khi:
*	Cả 2 bảng lớn
*	Không broadcast được

#### Quy trình:
1.	Shuffle cả 2 bảng
2.	Sort theo key
3.	Merge

👉 2 lần shuffle

👉 Rất tốn thời gian

---

### 3️⃣ ShuffleHashJoin (ít gặp)
* Shuffle 2 bảng
*	Build hash table

> ⚠️ Rủi ro memory cao → Spark ít chọn

---

### 4️⃣ Cartesian / Nested Loop (TỰ SÁT ☠️)

```python
df1.crossJoin(df2)
```

> 👉 Không bao giờ dùng trong bank

---

## 4️⃣ Vì sao JOIN nguy hiểm hơn GROUP BY?

|**GroupBy**|**Join**|
|-----------|--------|
|1 dataset	|2 dataset|
|1 shuffle	|2 shuffle|
|1 skew	|2 skew|
|Dễ đoán	|Rất khó đoán|

> 👉 Join = rủi ro nhân đôi

---

## 5️⃣ Ví dụ CHẾT NGƯỜI (thực tế bank)

```text
orders (2TB)
JOIN
customers (200GB)
ON customer_id
```

#### Nếu:
*	customers > broadcast threshold
*	key skew

#### 👉 Job:
*	Shuffle 2TB + 200GB
*	Runtime 3–5h
*	SLA vỡ
*	Incident 😱

---

## 6️⃣ spark.sql.autoBroadcastJoinThreshold

```python
spark.sql.autoBroadcastJoinThreshold = 10MB (default)
```

*	≤ threshold → Spark auto broadcast
*	threshold → SortMergeJoin

#### 📌 Bank thường:

```python
spark.sql.autoBroadcastJoinThreshold = -1
```

👉 Cấm auto broadcast

👉 Bắt architect quyết định

---

## 7️⃣ Đọc explain() khi join (BẮT BUỘC)

```python
orders.join(customers, "customer_id").explain()
```

#### Bạn PHẢI nhìn thấy:
*	BroadcastHashJoin ❓
*	SortMergeJoin ❗
*	Exchange ❗❗

---

## 8️⃣ JOIN AN TOÀN (Bank-grade)

### ✅ NÊN
*	Join dimension nhỏ
*	Join sau filter
*	Join sau pre-aggregation
*	Join theo key phân bố đều

```python
orders_filtered \
  .groupBy("customer_id", "date") \
  .sum("amount") \
  .join(customers_dim, "customer_id")
```

---

### ❌ TUYỆT ĐỐI TRÁNH
*	Join raw → raw
*	Join nhiều bảng lớn
*	Join trên key skew
*	Join trong Gold layer

---

## 9️⃣ Gold Layer & JOIN

### ❌ Sai:

```python
gold = orders.join(customers).join(accounts).join(products)
```

### ✅ Đúng:
*	Silver xử lý join nặng
*	Gold chỉ:
-	KPI
-	Snapshot
-	Aggregation nhẹ

---

### 🔟 CHECKPOINT A10 – PHẢI THUỘC
* Join = shuffle x2
*	Broadcast chỉ an toàn khi chắc chắn nhỏ
*	SortMergeJoin = mặc định & nguy hiểm
*	Join sai = không config nào cứu
*	Gold layer hạn chế join


# PHASE A11 – DATA SKEW (KẺ GIẾT SPARK TRONG IM LẶNG)

---

Nếu JOIN là “ác mộng công khai”
thì SKEW = ác mộng âm thầm, giết job khi mọi thứ tưởng ổn ☠️

---

# 📕 PHASE A11 – DATA SKEW (BANK-GRADE)

## 🎯 Mục tiêu A11

#### Sau bài này, bạn phải:
* Nhìn stage stuck 99% là biết skew
*	Biết vì sao retry vô dụng
*	Phân biệt salting vs AQE
*	Biết bank xử lý skew thế nào (không màu mè)

---

## 1️⃣ DATA SKEW LÀ GÌ?

**Skew** = phân bố dữ liệu **KHÔNG** đều

#### Ví dụ:

```text
customer_id = 1  →  25% dữ liệu
customer_id khác → 75% còn lại
```
👉 Khi groupBy / join theo key đó

👉 1 task phải xử lý khối dữ liệu khổng lồ

---

## 2️⃣ SKEW GIẾT SPARK NHƯ THẾ NÀO?

#### Ví dụ thực tế

```python
orders.groupBy("customer_id").sum("amount")
```

*	199 tasks xong trong 1–2 phút
*	1 task xử lý customer_id = 1
*	Task đó chạy 40 phút

> 👉 Stage stuck 99%

---

## 3️⃣ Vì sao 1 task giết cả stage?

Spark KHÔNG kết thúc stage cho tới khi task chậm nhất xong

*	Task ≠ chia nhỏ được khi đang chạy
*	Executor khác rảnh → lãng phí tài nguyên
*	SLA vỡ dù cluster còn dư

---

## 4️⃣ Vì sao RETRY KHÔNG CỨU ĐƯỢC SKEW?

#### ❌ Sai lầm phổ biến:

> “Retry lại job là xong”

#### ❗ Sự thật:
*	Retry = chạy lại Y NGUYÊN logic
*	Key skew vẫn skew
*	Task chậm vẫn chậm

> 👉 Skew là vấn đề DESIGN, không phải stability

---

## 5️⃣ Skew xuất hiện khi nào?

#### ⚠️ CỰC KỲ THƯỜNG GẶP
* groupBy(key)
*	join(key)
*	window partitionBy(key)
*	repartition(key)

---

## 6️⃣ Cách PHÁT HIỆN SKEW (BẮT BUỘC)

### 🔎 Spark UI

##### Vào:
* Stages
*	Xem:
-	Task time chênh lệch lớn
-	1 task chạy rất lâu
-	Shuffle Read cực lớn ở 1 task

> 👉 Đây là red flag số 1

---

## 7️⃣ SALTING – Cách cổ điển

#### Ý tưởng

##### Chia key lớn thành nhiều key nhỏ

```python
from pyspark.sql.functions import concat, lit, rand

orders_salted = orders.withColumn(
    "salted_key",
    concat(
        orders.customer_id,
        lit("_"),
        (rand() * 10).cast("int")
    )
)
```

##### Sau đó:
*	groupBy(salted_key)
*	aggregate
*	gộp lại

##### ⚠️ Nhược điểm
*	Code phức tạp
*	Khó maintain
*	Dễ sai logic

> 👉 Bank không thích salting nếu không bắt buộc

---

## 8️⃣ AQE – Adaptive Query Execution (Spark 3+)

```python
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.skewJoin.enabled", "true")
```

#### AQE làm gì?
* Detect skew khi runtime
*	Tự chia task lớn thành nhiều task nhỏ

👉 Ít code

👉 An toàn hơn salting

#### 📌 Nhưng:
* Không cứu được skew quá nặng
*	Không thay thế design tốt

---

## 9️⃣ Bank xử lý SKEW thế nào? (THỰC TẾ)

**Thứ tự ưu tiên:**

### 1️⃣ Fix design
* Tránh groupBy / join trên key skew
*	Pre-aggregate trước

### 2️⃣ Giảm data
* Filter sớm
*	Partition hợp lý

### 3️⃣ AQE
* Bật AQE, theo dõi

### 4️⃣ Salting
* CHỈ khi bắt buộc

### ❌ Không làm:
* Retry mù
*	Tăng executor vô hạn
*	“Cầu may”

---

## 🔟 CHECKPOINT A11 – PHẢI THUỘC
*	Skew = 1 key giết cả stage
*	Retry không cứu skew
*	Spark không chia task đang chạy
*	AQE > Salting (nếu dùng được)
*	Fix skew = fix design

---

## 🧠 CÂU HỎI BẮT BUỘC – A11 (TỰ TRẢ LỜI)
1.	Vì sao skew làm Spark chậm dù cluster còn rảnh?
2.	Vì sao retry không cứu được skew?
3.	Salting giải quyết skew bằng cách nào?
4.	AQE tốt hơn salting ở điểm nào?
5.	Khi nào KHÔNG nên dùng salting?



# PHASE A12 – MEMORY & EXECUTOR (ẢO TƯỞNG “CỨ TĂNG RAM LÀ NHANH”)

---

PHASE A (FOUNDATION) có tổng cộng 15 bài

### Bạn đang ở:
*	✅ A11 – Data Skew
*	👉 A12 – Memory & Executor (hiện tại)
*	🔜 A13 – Partition & File layout
*	🔜 A14 – Spark UI đọc như bác sĩ
*	🔜 A15 – Tổng hợp tư duy Spark Architect

> 👉 Sau A15 là PHASE B – Spark thực chiến + tuning bài bản

---

# 📕 PHASE A12 – MEMORY & EXECUTOR (BANK-GRADE)

## 🎯 Mục tiêu A12

### Sau bài này bạn phải:
*	Biết vì sao tăng executor memory KHÔNG luôn cứu job
*	Hiểu executor = JVM process, không phải “cục RAM”
*	Biết spill to disk là gì & khi nào xảy ra
*	Không còn tuning kiểu “cầu may”

---

## 1️⃣ Executor thực chất là gì?

Executor = 1 JVM process chạy trên worker node

#### Bên trong executor có:
*	JVM Heap (Spark memory)
*	Python worker (nếu PySpark)
*	Native libs
*	Shuffle buffer
*	Cache / Broadcast

> 👉 Không phải toàn bộ RAM bạn cấp đều dùng cho data

---

## 2️⃣ Spark chia memory như thế nào? (rất quan trọng)

Tổng thể (đơn giản hóa):

```text
Executor Memory
├── Spark Memory
│   ├── Execution (shuffle, join, agg)
│   └── Storage (cache)
├── User Memory
└── Reserved (overhead)
```

> ⚠️ Execution luôn ưu tiên hơn Storage

>- 👉 Cache có thể bị đẩy ra (evict) bất kỳ lúc nào

---

## 3️⃣ Vì sao executor quá to lại PHẢN TÁC DỤNG?

### ❌ Suy nghĩ sai:

“Executor càng to càng nhanh”

#### Thực tế:
* JVM lớn → GC chậm
*	Một task lỗi → mất nhiều tài nguyên
*	Ít executor → ít song song
*	Broadcast / shuffle buffer lớn → dễ OOM

> 👉 Bank không bao giờ dùng executor quá to

#### 📌 Thực chiến:
* 4–6 cores / executor
*	Memory vừa phải, nhiều executor

---

## 4️⃣ Khi nào nên CACHE? Khi nào KHÔNG?

### ✅ Nên cache khi:
*	Dataset được dùng lại ≥ 2 lần
*	Sau filter / reduce lớn
*	Fit vào memory

### ❌ Không cache khi:
*	Dataset chỉ dùng 1 lần
*	Dataset rất lớn
*	Job có nhiều shuffle nặng

> ⚠️ Cache sai = job chậm hơn

---

## 5️⃣ SPILL TO DISK là gì?

Khi Spark không đủ memory, nó ghi tạm data ra disk

#### Xảy ra khi:
*	Shuffle lớn
*	GroupBy nhiều key
*	Join lớn
*	Partition quá to

> 👉 Disk = chậm hơn RAM hàng chục lần

---

## 6️⃣ Vì sao tăng executor memory KHÔNG cứu được job?

Vì:

*	Skew vẫn skew
*	Shuffle vẫn lớn
*	Partition vẫn to
*	Design vẫn sai

> 👉 Memory chỉ che triệu chứng, không chữa bệnh

---

## 7️⃣ spark.executor.cores bao nhiêu là hợp lý?

#### 📌 Bank-grade rule of thumb:

* 2–4 cores / executor
* Memory vừa đủ
* Nhiều executor > executor khổng lồ

Vì:

*	GC nhanh
*	Parallel tốt
*	Failure impact nhỏ

---

## 8️⃣ Dấu hiệu job đang “chết vì memory”

Trong Spark UI:

*	GC Time cao
*	Spill (memory/disk) lớn
*	Task OOM
*	Stage chạy lâu dù CPU thấp

> 👉 Đây là design smell, không phải config thiếu

---

## 9️⃣ CHECKPOINT A12 – PHẢI NHỚ
*	Executor = JVM process
*	Memory chia nhiều vùng, không phải toàn bộ cho data
*	Executor to ≠ job nhanh
*	Cache sai còn tệ hơn không cache
*	Memory không cứu design sai

---

## 🧠 CÂU HỎI BẮT BUỘC – A12 (TỰ TRẢ LỜI)
1.	Vì sao executor quá to lại phản tác dụng?
2.	Khi nào nên cache, khi nào KHÔNG?
3.	Spill to disk xảy ra khi nào?
4.	Vì sao tăng memory không luôn giải quyết chậm?
5.	spark.executor.cores nên bao nhiêu là hợp lý?


# PHASE A13 – PARTITION & FILE LAYOUT

(đây là bài cực kỳ quan trọng, vì 80% job Spark chậm là do partition sai, không phải code)

---

# 📕 PHASE A13 – PARTITION & FILE LAYOUT

“Nhỏ quá thì overhead – To quá thì spill – Vừa sai là chết”

---

## 🎯 Mục tiêu A13

Sau bài này bạn phải:

*	Hiểu partition là gì (thật sự)
*	Biết vì sao số file ảnh hưởng trực tiếp đến cost & SLA
*	Phân biệt:
-	repartition
-	coalesce
-	spark.sql.shuffle.partitions
-	Biết file layout chuẩn bank-grade

---

## 1️⃣ Partition là gì? (nói cho đúng)

**Partition** = đơn vị song song nhỏ nhất của **Spark**

*	1 partition → 1 task
*	1 task → chạy trên 1 core
*	Không thể chia nhỏ partition khi đang chạy

> 👉 Spark nhanh hay chậm = partition hợp lý hay không

---

## 2️⃣ Partition quá ÍT → chuyện gì xảy ra?

#### Ví dụ:
* 1 TB data
*	10 partitions

#### ⛔ Hậu quả:
*	Mỗi task gánh 100 GB
*	Spill to disk
*	Task chạy rất lâu
*	Stage stuck 99%

> 👉 CPU còn rảnh nhưng job vẫn chậm

---

## 3️⃣ Partition quá NHIỀU → chuyện gì xảy ra?

#### Ví dụ:
*	1 GB data
*	10,000 partitions

#### ⛔ Hậu quả:
*	Task quá nhỏ
*	Scheduler overhead lớn
*	Shuffle metadata khổng lồ
*	HDFS / GCS nhiều file nhỏ → cực chậm

> 👉 “Small file problem”

---

## 4️⃣ Con số VÀNG: partition bao nhiêu là vừa?

### 📌 Rule of thumb (bank-grade):

```text
1 partition ≈ 100MB – 300MB
```

#### Ví dụ:
*	100 GB data → ~300–800 partitions

> ⚠️ Không phải luật cứng, nhưng rất thực tế

---

## 5️⃣ spark.sql.shuffle.partitions là gì?

***Số partition sau mỗi shuffle***

#### Mặc định:

```python
spark.sql.shuffle.partitions = 200
```

> ***⛔ Với dataset nhỏ → rất nguy hiểm***

#### 📌 Best practice:
*	Dataset nhỏ: giảm (20–50)
*	Dataset lớn: set theo data size

> 👉 Không để default trong production

---

## 6️⃣ repartition vs coalesce (rất hay bị nhầm)

### 🔁 repartition(n)
*	Có shuffle
*	Chia lại data đều
*	**Dùng khi:**
-	Trước join
-	Trước agg
-	Fix skew

```python
df = df.repartition(200)
```
---

### 🔽 coalesce(n)
* KHÔNG shuffle
*	Gộp partition hiện có
*	Chỉ giảm partition
*	**Dùng khi:**
-	Trước write
-	Giảm file output

```python
df = df.coalesce(20)
```

> 📌 ***coalesce*** không dùng để ***fix*** **skew**

---

## 7️⃣ File layout – Vì sao ngân hàng rất khắt khe?

### ❌ Layout tệ:
*	Hàng triệu file nhỏ
*	Không partition theo date
*	Không predictable

### ✅ Layout chuẩn bank-grade:

```text
/gold/orders/
  └── dt=2025-01-05/
      ├── part-00000.parquet
      ├── part-00001.parquet
```

### Partition theo:
*	Date (dt)
*	Country
*	Business dimension ổn định

---

## 8️⃣ Vì sao Gold layer cực kỳ ghét file nhỏ?

*	Query BI chậm
*	Cost tăng
*	SLA không ổn định
*	Audit khó

> 👉 Gold = ít file, rõ ràng, predictable

---

## 9️⃣ Dấu hiệu partition đang SAI

### Trong Spark UI:
*	Task time chênh lệch lớn
*	Shuffle spill cao
*	Stage stuck
*	Output hàng chục nghìn file

> 👉 Đây là lỗi design, không phải infra

---

## 🔥 CHECKPOINT A13 – PHẢI NHỚ

*	Partition = task = core
*	Quá ít → spill
*	Quá nhiều → overhead
*	repartition ≠ coalesce
*	Không bao giờ để shuffle.partitions mặc định trong prod
*	File layout = performance + cost + audit

---

## 🧠 CÂU HỎI BẮT BUỘC – A13

1.	Vì sao partition quá ít làm job chậm dù CPU rảnh?
2.	Vì sao file nhỏ gây hại trong Data Lake?
3.	Khi nào dùng repartition, khi nào dùng coalesce?
4.	Vì sao spark.sql.shuffle.partitions=200 nguy hiểm?
5.	Vì sao Gold layer cần layout cực kỳ chuẩn?


# PHASE A14 – SPARK UI: ĐỌC NHƯ BÁC SĨ (5 PHÚT BIẾT JOB SỐNG HAY CHẾT)

---

## 📕 PHASE A14 – SPARK UI (READ LIKE A DOCTOR)

---

## 🎯 Mục tiêu A14

#### Sau bài này bạn phải:

*	Mở Spark UI và biết nhìn chỗ nào trước
*	**Phân biệt:**
-	Job chậm vì data
-	Job chậm vì design
-	Job chậm vì config
-	Biết khi nào nên kill job ngay

---

## 1️⃣ Spark UI gồm những tab nào? (chỉ cần nhớ 5)

|**Tab**|**Mục đích**|
|-------|------------|
|Jobs|Tổng quan job|
|Stages|	Nơi đọc bệnh|
|Tasks|	Chi tiết từng task|
|SQL|	Logical / Physical plan|
|Executors|	Memory + CPU|

> 👉 90% thời gian bạn ở Stages + Tasks

---

## 2️⃣ Trình tự đọc Spark UI (RẤT QUAN TRỌNG)

#### ❌ Sai lầm phổ biến:
*	Nhảy vào code
*	Tăng memory
*	Retry job

#### ✅ Trình tự chuẩn:

```text
Jobs → Stages → Tasks → Executors → SQL
```

---

## 3️⃣ Jobs tab – nhìn gì?

#### Trong Jobs:
*	Job nào lâu nhất?
*	Job có retry không?
*	Job có action nào?

> 👉 Job chỉ là “triệu chứng”, chưa phải nguyên nhân

---

## 4️⃣ Stages tab – NƠI BÁC SĨ ĐỨNG

### 🔍 Nhìn 5 thứ bắt buộc:
1.	Duration
2.	Shuffle Read / Write
3.	Input / Output size
4.	Tasks completed
5.	Stage stuck %

---

### 🚨 DẤU HIỆU NGUY HIỂM

|**Hiện tượng**|**Ý nghĩa**|
|--------------|-----------|
|Stage stuck 99%|	Skew|
|Shuffle Read lớn	|Join / agg nặng|
|Duration tăng dần	|Spill|
|Stage retry|	Memory / OOM|
|1 stage chiếm 80% runtime	|Design sai|


---

## 5️⃣ Tasks tab – soi như kính hiển vi

### 🔍 Nhìn những cột này:
*	Duration
*	Input size
*	Shuffle Read
*	Spill (Memory / Disk)

---

### 🚨 SKew điển hình

#### Ví dụ:
*	95% task: 1–2s
*	1 task: 15 phút

> 👉 Chắc chắn skew

>> 📌 Retry KHÔNG cứu được

---

## 6️⃣ Executors tab – đừng nhìn mỗi memory

### Nhìn:
*	Active tasks
*	Failed tasks
*	GC time
*	Spill

### 🚫 Sai lầm

```text
“Executor memory còn dư → job không thiếu RAM”
```

### ❗ Sai:
*	JVM heap ≠ usable memory
*	Python / shuffle buffer ăn rất nhiều

---

## 7️⃣ SQL tab – đọc plan thế nào cho đúng?

Không đọc hết.

### Chỉ cần tìm:
*	Exchange
*	Join type
*	BroadcastExchange
*	SortMergeJoin

> 📌 Exchange = shuffle = cost

---

## 8️⃣ Khi nào PHẢI kill job ngay?

### ⛔ Kill ngay nếu:
*	Stage stuck > 95%
*	Spill liên tục
*	Shuffle > kỳ vọng
*	Broadcast OOM
*	SLA chắc chắn vỡ

> 👉 Kill sớm = cứu cluster + cứu cost

---

## 9️⃣ Spark UI giúp bạn quyết định điều gì?

|**Thấy gì**|**Quyết định**|
|-----------|--------------|
|Skew|Salting / AQE|
|Shuffle lớn|Redesign join|
|Nhiều file nhỏ|coalesce|
|Task imbalance|repartition|
|GC cao|executor config|


---

## 🔥 CHECKPOINT A14 – GHI NHỚ
*	Spark UI > Code
*	Stage là nơi đọc bệnh
*	Skew = design problem
*	Retry không cứu skew
*	Kill job sớm là quyết định GIỎI

---

## 🧠 CÂU HỎI BẮT BUỘC – A14
1.	Vì sao Stage tab quan trọng hơn Job tab?
2.	Stage stuck 99% thường do đâu?
3.	Vì sao nhìn executor memory chưa đủ?
4.	Khi nào kill job là quyết định đúng?
5.	Exchange trong SQL plan nói lên điều gì?



# PHASE A15 – SHUFFLE & SKEW: KẺ GIẾT SPARK THẦM LẶNG

---

# 🔥 PHASE A15 – SHUFFLE & DATA SKEW

---

## 🎯 Mục tiêu A15

#### Sau bài này bạn phải:
*	Hiểu shuffle là gì ở mức vật lý
*	Biết vì sao shuffle đốt CPU + disk + network
*	Nhận diện skew chỉ bằng Spark UI
*	Biết khi nào config vô nghĩa, phải sửa design

---

## 1️⃣ Shuffle là gì? (KHÔNG DÙNG ĐỊNH NGHĨA SÁCH)

### ❌ Định nghĩa sai thường gặp

Shuffle là Spark chia lại dữ liệu

### ✅ Định nghĩa đúng (architect-level)

Shuffle = Spark phải ghi dữ liệu ra disk + gửi qua network + đọc lại để tiếp tục xử lý

#### 📌 Mỗi shuffle gồm:
*	Serialize
*	Write disk
*	Network transfer
*	Read disk
*	Deserialize

> 👉 5 bước – step nào cũng tốn tiền

---

## 2️⃣ Những operation nào BẮT BUỘC shuffle?

|**Operation**|**Shuffle?**|
|-------------|------------|
|groupBy|	✅|
|join (không broadcast)	|✅|
|distinct	|✅|
|orderBy|	✅|
|repartition|	✅|
|coalesce	|❌|

> 👉 Nếu bạn thấy nhiều shuffle → design đang có vấn đề

---

## 3️⃣ Vì sao shuffle giết Spark?

|**Resource**|**Bị ăn**|
|------------|---------|
|CPU|serialize / sort|
|Disk|spill|
|Network|cross-node|
|Memory|buffer|

> 📌 Tăng RAM chỉ cứu 1 phần – không cứu design

---

## 4️⃣ Skew là gì? (cách hiểu ĐÚNG)

### ❌ Sai:

Skew là dữ liệu không đều

### ✅ Đúng:

Skew = một partition nhận workload vượt xa các partition khác

> 📌 Skew = task imbalance, không phải record imbalance đơn thuần

---

## 5️⃣ Skew biểu hiện trong Spark UI thế nào?

### Stage tab:
* Stage stuck 99%
*	1 task chạy rất lâu

### Tasks tab:
*	1 task duration >> others
*	Shuffle read cực lớn ở 1 task

> 👉 Đây là dấu hiệu GIẾT job

---

## 6️⃣ Vì sao retry KHÔNG cứu được skew?

|**Retry**|**Kết quả**|
|Lần 1|Fail|
|Lần 2|Fail|
|Lần 10|Fail|

### 👉 Vì:
*	Data không đổi
*	Partitioning không đổi
*	Task nặng vẫn nặng

> 📌 Retry chỉ dành cho lỗi hạ tầng, không dành cho skew

---

## 7️⃣ Những nguyên nhân skew phổ biến (BANK THƯỜNG GẶP)

*	1 customer_id chiếm 30% data
* status = “SUCCESS” chiếm 90%
*	date cuối tháng
*	NULL key
*	join key không phân bố đều

> 👉 80% skew đến từ business data

---

## 8️⃣ Cách xử lý skew – phân cấp tư duy

### 🥉 Level thấp – Config (chỉ cứu nhẹ)
*	spark.sql.shuffle.partitions
*	AQE

> 👉 Không giải quyết gốc

---

### 🥈 Level trung – Salting
*	Chia key lớn thành nhiều key nhỏ
*	Hiệu quả nhưng phức tạp

---

### 🥇 Level cao – DESIGN
*	Tách pipeline
*	Pre-aggregate
*	Change join strategy
*	Refactor Gold/Silver boundary

> 👉 Design > config

---

## 9️⃣ AQE xử lý skew như thế nào?

### AQE có thể:
*	Detect skew partition
*	Split partition lớn thành nhỏ

### 📌 Nhưng:
* Không cứu được skew cực lớn
*	Không thay đổi business logic

> 👉 AQE là cứu trợ, không phải bác sĩ

---

## 🔥 CHECKPOINT A15 – GHI NHỚ

*	Shuffle = disk + network + money
*	Skew = task imbalance
*	Retry không cứu skew
*	Config không cứu design
*	Spark UI luôn nói thật

---

## 🧠 CÂU HỎI BẮT BUỘC – A15
1.	Vì sao shuffle luôn đắt hơn compute?
2.	Skew khác imbalance ở điểm nào?
3.	Vì sao retry không cứu skew?
4.	AQE giúp được gì và KHÔNG giúp được gì?
5.	Khi nào buộc phải redesign pipeline?


# 🧠 A16 – PARTITION STRATEGY (BÀI CHỐT LÝ THUYẾT)

90% Spark job chết không phải vì code, mà vì partition sai

--

## 1️⃣ Partition là gì? (hiểu cho đúng)

Partition = đơn vị xử lý song song nhỏ nhất của Spark

*	1 partition → 1 task
*	1 task → chạy trên 1 core
*	Không có partition tốt → cluster mạnh cũng vô nghĩa

> 👉 Spark không scale bằng data, Spark scale bằng partition

---

## 2️⃣ Các loại partition trong Spark

### 🔹 Input Partition
####	Khi đọc từ:
*	HDFS
*	GCS / S3
*	Parquet / ORC

#### Ví dụ:
*	100 file Parquet → ~100 partitions
*	1 file lớn → 1 partition (❌ nguy hiểm)

---

### 🔹 Shuffle Partition (nguy hiểm nhất)
####	Tạo khi:
*	groupBy
*	join
*	distinct
*	orderBy

#### 👉 Controlled bằng:

```python
spark.sql.shuffle.partitions
```

> ⚠️ Default = 200 (rất hay giết job nhỏ)

---

## 3️⃣ Repartition vs Coalesce (tóm gọn chuẩn thi kiến trúc)

### 🔄 repartition()
*	Có shuffle
*	Tăng hoặc giảm partition
*	Chậm nhưng cân bằng

### Dùng khi:
*	Trước join
*	Trước write lớn
*	Sau skew fix

---

### 🔽 coalesce()
*	KHÔNG shuffle
*	Chỉ giảm partition
*	Nhanh nhưng dễ lệch

### Dùng khi:
* Trước write cuối
*	Sau filter lớn

---

## 4️⃣ Partition key – chọn sai là chết

### ❌ Key tệ
*	status (SUCCESS / FAILED)
*	country (VN, SG)
*	gender (M/F)

> → Cardinality thấp → skew

---

### ✅ Key tốt
*	customer_id (phân bố đều)
*	order_id
*	hash(key)

---

## 5️⃣ Partition theo DATA ≠ Partition theo SLA

#### Partition theo data

*	File size đều
*	Phân bố đẹp
*	Nhưng không chắc nhanh

#### Partition theo SLA

Nhắm:

*	Task runtime < 5–10 phút
* Không spill
*	Không skew

> 👉 BANK chọn SLA, không chọn đẹp

---

## 6️⃣ Dấu hiệu partition sai (đọc UI là thấy)

*	Stage stuck 99%
*	Một task chạy gấp 10–100 lần task khác
*	Shuffle Read rất lớn
*	Spill to disk liên tục

> 👉 Fix bằng design, không phải config

---

## 7️⃣ Checklist partition chuẩn BANK

* ✔ Partition đủ để dùng hết core
* ✔ Task runtime < 5–10 phút
* ✔ Không key skew
* ✔ Shuffle có kiểm soát
* ✔ Gold layer partition ổn định (theo date, snapshot)

---

## 🧠 CÂU HỎI BẮT BUỘC – A16

#### Trả lời bằng suy nghĩ, không cần code:

1.	Vì sao tăng partition quá nhiều lại làm job chậm?
2.	Khi nào nên repartition trước join?
3.	Vì sao Gold layer KHÔNG nên repartition theo customer?
4.	Vì sao coalesce sai có thể gây OOM khi write?
5.	Partition nên thiết kế theo data hay SLA?


---

## 🚨 QUAN TRỌNG

### Sau A16:

* ❌ Không còn lý thuyết nữa
*	🔥 Vào THỰC CHIẾN SPARK
*	🧪 Case thật
*	💣 Incident thật
*	🏦 Mindset BANK
